In [51]:
from typing import List, Tuple, Dict, Any, Union

import pandas as pd
import streamlit as st

# Documentação

## Anotação de tipos

- Jeito super fácil de documentar funções
- Evita adivinhar os tipos dos argumentos e do retorno
- Evita ler o código para inferir os tipos
- Type hints cheat sheet (5 min, ler até functions).


In [ ]:
def calcula_cotacoes_periodo(
    index: pd.DataFrame,
    id_titulo: int, 
    data_inicio: pd.Timestamp, 
    data_fim: pd.Timestamp, 
    taxa: float, 
    valor: float,
) -> pd.Series:
    
    mask_periodo = (index.index >= data_inicio) & (index.index < data_fim)
    fator_cdi = index.loc[mask_periodo, "Fator CDI"] * taxa + 1
    cotacoes = fator_cdi.cumprod() * valor
    return (
        cotacoes.reset_index()
        .rename(columns={"Fator CDI": "Cotação"})
        .assign(ID=id_titulo)
    )

## Docstrings

- Útil para documentar funções complexas ou abstratas.
- Use o Chat GPT para autocompletar os argumentos e retorno.

In [4]:
def calcular_sharpe_ratio(
    series_precos: pd.Series, 
    taxa_livre_risco: float
) -> float:
    """
    Calcula o Índice de Sharpe para uma série de preços de ações.

    Args:
        series_precos (pd.Series): Uma série contendo os preços 
            diários de uma ação.
        taxa_livre_risco (float): A taxa de retorno livre de risco, 
            geralmente o retorno de um título do governo.

    Returns:
        float: O Índice de Sharpe calculado.
    """
    
    retornos = series_precos.pct_change().dropna()
    retorno_medio = retornos.mean() * 252
    desvio_padrao = retornos.std() * (252 ** 0.5)

    sharpe_ratio = (retorno_medio - taxa_livre_risco) / desvio_padrao
    return sharpe_ratio

## Docstrings nem sempre são necessárias

Funções “anotadas” e com nomes auto-explicativos não precisam de docstring.

In [5]:
def calcular_sharpe_ratio(
    series_precos: pd.Series, 
    taxa_livre_risco: float
) -> float:
    
    retornos = series_precos.pct_change().dropna()
    retorno_medio = retornos.mean() * 252
    desvio_padrao = retornos.std() * (252 ** 0.5)

    sharpe_ratio = (retorno_medio - taxa_livre_risco) / desvio_padrao
    return sharpe_ratio

# Melhorando legibilidade e entendimento

## Evite comentários óbvios ou que "narram" o código

In [ ]:
def calcular_sharpe_ratio(series_precos: pd.Series, taxa_livre_risco: float) -> float:
    """
    Calcula o Índice de Sharpe para uma série de preços de ações.

    Args:
        series_precos (pd.Series): Uma série contendo os preços diários de uma ação.
        taxa_livre_risco (float): A taxa de retorno livre de risco, geralmente o retorno de um título do governo.

    Returns:
        float: O Índice de Sharpe calculado.
    """
    
    retornos = series_precos.pct_change().dropna()  # Calcula os retornos diários, ignorando o primeiro NaN
    retorno_medio = retornos.mean() * 252  # Média dos retornos diários multiplicada pelo número de dias de negociação em um ano
    desvio_padrao = retornos.std() * (252 ** 0.5)  # Desvio padrão dos retornos diários anualizado

    sharpe_ratio = (retorno_medio - taxa_livre_risco) / desvio_padrao
    return sharpe_ratio

## Use comentários para explicar manipulações ou contextos complexos

## Números mágicos

- Crie constantes com nomes que indiquem o significado de números mágicos.
- Outra opção adicionar um comentário explicando o número. 

## Formatação

- PEP 8: documento que direciona como código Python deve ser formatado.
- flake8: pacote que valida a PEP 8 e ainda traz indicações de falhas de segurança, expressões complexas e pedaços de código não idiomático.
- black: pacote para padronizar a formatação do seu código **automaticamente**.

# Dicas sobre estrutura do código

## Procure não repetir código

In [52]:
df = pd.DataFrame({
    "ID": [1, 1, 1, 1, 1],
    "Data": ["2021-01-01", "2021-01-02", "2021-01-03", "2021-01-04", "2021-01-05"],
    "Cotação": [100, 101, 102, 103, 104],
    "Fator CDI": [1.0001, 1.0002, 1.0003, 1.0004, 1.0005],
    "Imposto": [0.1, 0.1, 0.1, 0.1, 0.1],
})

In [53]:
df["ID"] = pd.to_numeric(df["ID"])
df["Cotação"] = pd.to_numeric(df["Cotação"])
df["Fator CDI"] = pd.to_numeric(df["Fator CDI"])
df["Imposto"] = pd.to_numeric(df["Imposto"])
df["Data"] = pd.to_datetime(df["Data"])

In [ ]:
for col in ["ID", "Cotação", "Fator CDI", "Imposto"]:
    df[col] = pd.to_numeric(df[col])

df["Data"] = pd.to_datetime(df["Data"])

## Faça validações no começo e termine o fluxo rápido

In [ ]:
def pagina_busca():
    st.title("Busca de Títulos Públicos")

    id_titulo = st.number_input("ID do Título", min_value=1, max_value=9999, value=1)
    data_inicio = st.date_input("Data de Início", pd.Timestamp("2021-01-01"))

    if id_titulo > 0:
        if data_inicio >= pd.Timestamp("2021-01-01"):
            st.write(df)
        else:
            st.error("A data de início deve ser a partir de 01/01/2021.")
    else:
        st.error("O ID do Título deve ser um número positivo.")

In [ ]:
def pagina_busca():
    st.title("Busca de Títulos Públicos")

    id_titulo = st.number_input("ID do Título", min_value=1, max_value=9999, value=1)
    data_inicio = st.date_input("Data de Início", pd.Timestamp("2021-01-01"))

    if id_titulo < 0:
        st.error("O ID do título deve ser um número positivo.")
        return
    elif data_inicio < pd.Timestamp("2021-01-01"):
        st.error("A data de início deve ser a partir de 01/01/2021.")
        return
    else:
        st.write(df)

In [ ]:
def pagina_busca():
    st.title("Busca de Títulos Públicos")

    id_titulo = st.number_input("ID do Título", min_value=1, max_value=9999, value=1)
    data_inicio = st.date_input("Data de Início", pd.Timestamp("2021-01-01"))

    if id_titulo < 0:
        st.error("O ID do título deve ser um número positivo.")
        return
    
    if data_inicio < pd.Timestamp("2021-01-01"):
        st.error("A data de início deve ser a partir de 01/01/2021.")
        return
    
    # A página continua aqui sem precisar identar o código
    st.write(df)

# Padrões e boas práticas no Python

- Fazer as coisas de forma **"pythonica"**.

## Use context manager sempre que possível

Evite:

In [47]:
file_pointer = open("hash_table.png", "rb")
print(file_pointer.read()[:10])
file_pointer.close()

b'\x89PNG\r\n\x1a\n\x00\x00'


Use: 

In [49]:
with open("hash_table.png", "rb") as file_pointer:
    print(file_pointer.read()[:10])

b'\x89PNG\r\n\x1a\n\x00\x00'


## Use dicionários sempre que puder

- Por baixo dos panos é uma estrutura de dados chamada Hash Table.
- Permite obter informações de forma bem rápida usando uma função Hash.

<img src="hash_table.png" width="700">

In [2]:
target = "James"
phone_numbers_list = [("James", "555-5555"), ("Ellen", "555-1234"), ("Bill", "555-4444")]
phone_numbers_dict = {"James": "555-5555", "Ellen": "555-1234", "Bill": "555-4444"}

In [4]:
%%timeit
for name, number in phone_numbers_list:
    if name == target:
        break

47.1 ns ± 0.636 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [5]:
%%timeit
phone_numbers_dict["James"]

17.9 ns ± 0.0583 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


## Use conjuntos sempre que puder

- Ordem não importa
- Semelhante à Hash Table, mas sem o valor.
- Faz testes de pertencimento muito rápido.

In [6]:
items = list(range(1000))
conjunto = set(items)

In [7]:
%%timeit
100 in items

442 ns ± 6.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
%%timeit
100 in conjunto

18.4 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


## F-string

- Mais curto e legível do que outras formas.

In [11]:
nome = "Alice"
idade = 30

In [12]:
saudacao = "Olá, " + nome + "! Você tem " + str(idade) + " anos."
saudacao = f"Olá, {nome}! Você tem {idade} anos."

## Desempacotar tuplas

In [13]:
valores = ("Mateus", "DRRCA", "Cientista de Dados")

In [14]:
nome = valores[0]
area = valores[1]
cargo = valores[2]

In [15]:
nome, area, cargo = valores

## Estruturas vazias

- Funciona para listas, conjuntos, dicionários, tuplas.

In [17]:
valores = [1, 2, 3]

In [18]:
if len(valores) > 0:
    print("A lista não está vazia!")

A lista não está vazia!


In [19]:
if valores:
    print("A lista não está vazia!")

A lista não está vazia!


## Loops baseado em índices

- Não devem ser feitos.

In [21]:
deslocamentos = [234, 234234, 39]

for distancia in deslocamentos:
    print(f"O carro andou {distancia} metros.")

O carro andou 234 metros.
O carro andou 234234 metros.
O carro andou 39 metros.


In [22]:
for idx, distancia in enumerate(deslocamentos):
    print(f"{idx}: o carro andou {distancia} metros.")

0: o carro andou 234 metros.
1: o carro andou 234234 metros.
2: o carro andou 39 metros.


In [23]:
coordenadas = [(1, 2), (3, 4), (5, 6)]
for x, y in coordenadas:
    print(f"({x}, {y})")

(1, 2)
(3, 4)
(5, 6)


In [25]:
coordenadas = [(1, 2), (3, 4), (5, 6)]
for idx, (x, y) in enumerate(coordenadas):
    print(f"{idx}: ({x}, {y})")

0: (1, 2)
1: (3, 4)
2: (5, 6)


In [26]:
distancia = [234, 234234, 39]
coordenadas = [(1, 2), (3, 4), (5, 6)]

for d, (x, y) in zip(distancia, coordenadas):
    print(f"O carro andou {d} metros e está na posição ({x}, {y}).")

O carro andou 234 metros e está na posição (1, 2).
O carro andou 234234 metros e está na posição (3, 4).
O carro andou 39 metros e está na posição (5, 6).


## Lops em dicionários

In [30]:
exemplo = {"nome": "Alice", "idade": 30, "cargo": "Cientista de Dados"}
for chave in exemplo.keys():
    print(f"{chave}: {exemplo[chave]}")

nome: Alice
idade: 30
cargo: Cientista de Dados



- Podemos iterar nas chaves diretamente

In [27]:
for chave in exemplo:
    print(chave)

nome
idade
cargo


- Podemos iterar nas chaves e valores ao mesmo tempo

In [28]:
for chave, valor in exemplo.items():
    print(f"{chave}: {valor}")

nome: Alice
idade: 30
cargo: Cientista de Dados


## List comprehension

- Usar quando precisar criar estruturas de repetição simples para criar listas, dicionários e conjuntos.

In [31]:
[f"Nome: {nome}" for nome in ["Alice", "Bob", "Charlie"]]

['Nome: Alice', 'Nome: Bob', 'Nome: Charlie']

In [33]:
[numero for numero in range(10) if numero % 2 == 0]

[0, 2, 4, 6, 8]

In [34]:
{numero: numero ** 2 for numero in range(10)}

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49, 8: 64, 9: 81}

In [37]:
{numero ** 2 for numero in range(10)}

{0, 1, 4, 9, 16, 25, 36, 49, 64, 81}

Mas não exagere kkk

In [41]:
combinacoes = [
    (x, y, z) 
    for x in range(3) 
    for y in range(3) 
    for z in range(3)
]
combinacoes

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 2, 0),
 (2, 2, 1),
 (2, 2, 2)]

## Defina sempre None como valor padrão de um parâmetro opcional

Comportamento estranho:

In [42]:
def foo(numero, lista=[]):
    lista.append(numero)
    return lista

print(foo(1))
print(foo(2))

[1]
[1, 2]


Correto:

In [43]:
def foo(numero, lista=None):
    if not lista:
        lista = []
    lista.append(numero)
    return lista

print(foo(1))
print(foo(2))

[1]
[2]


## Evite loops ao usar pandas/numpy

- É possível fazer quase tudo nesses pacotes sem um **for**.

# Gestão de código

- Remova sem dó código antigo ou que você pode ser que use no futuro. Ele está salvo no Git.
- Use branches no git.